In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import xgboost as xg

visit_id | ---UniProt (~218)--- | updrs_x

In [20]:
df_pivot

NameError: name 'df_pivot' is not defined

In [21]:
def read_data():
    train_clinical_data = pd.read_csv('data/train_clinical_data.csv')[
    ['visit_id', 'patient_id', 'visit_month', 'upd23b_clinical_state_on_medication', 'updrs_1', 'updrs_2','updrs_3', 'updrs_4']]
    train_proteins = pd.read_csv('data/train_proteins.csv')
    return train_clinical_data, train_proteins

In [22]:
def data_transform(df):
    # Applying log transformation
    df.iloc[:, 1:-2] = df.iloc[:, 1:-2].apply(np.log)
    df = df.replace(float("-inf"), 0)
    

    #normalizing to (0,1)
    scaler = MinMaxScaler()
    scaler.fit(df.iloc[:, 1:-2])
    transformed_values = scaler.transform(df.iloc[:, 1:-2])
    df.iloc[:, 1:-2] = transformed_values

    #increasing the spread
    df.iloc[:, 1:-2] = df.iloc[:, 1:-2].apply(lambda x : np.power(x, 4)) 
    df = df.round(4)
     
    return df

In [51]:
def data_prep(train_clinical_data, train_proteins):
    
    df = train_clinical_data.merge(train_proteins, how='inner', on='visit_id')
    df.head()
    df_pivot = df.pivot(index=['visit_id', 'visit_month_x'], values='NPX', columns=['UniProt'])
    df_pivot.fillna(0, inplace=True)
    df_updrs = df[['visit_id', 'visit_month_x','updrs_1']].groupby(by='visit_id').mean().reset_index()
    df_pivot = df_pivot.merge(df_updrs, how='left', on='visit_id')

    df_pivot = data_transform(df_pivot)
    X_train, X_test, y_train, y_test = train_test_split(df_pivot[df_pivot.columns.difference(['visit_id', 'updrs_1'])], 
                                                    df_pivot[['updrs_1']], test_size=0.1, shuffle=True, random_state=42)
    return X_train, X_test, y_train, y_test

In [52]:
def train(model, X_train, y_train):
    model.fit(X_train, y_train.squeeze())
    return model

In [53]:
def predict(model, X_test):
    y_pred = model.predict(X_test)
    return y_pred

In [54]:
def pipeline():
    train_clinical_data, train_proteins = read_data()
    X_train, X_test, y_train, y_test = data_prep(train_clinical_data, train_proteins)
    model = xg.XGBRegressor(objective ='reg:linear', )
    model = train(model, X_train, y_train)
    y_pred = predict(model, X_test)
    return {
        "model" : model,
        "X_train" : X_train,
        "X_test" : X_test, 
        "y_train" : y_train, 
        "y_test" : y_test,
        "y_pred" : y_pred
    }

In [55]:
results = pipeline()

[12:33:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [57]:
mean_squared_error(results['y_pred'], results['y_test'], squared = False)

3.9361535613465644

In [39]:
results['y_test']

,updrs_2
707,0.0
215,0.0
881,3.0
88,1.0
784,0.0
...,...
885,3.0
825,5.0
210,0.0
1020,17.0


In [ ]:
np.log10(df_pivot['O00391'])

In [ ]:
plt.plot(df_pivot['O00391'], df_pivot['updrs_1'])

In [ ]:
plt.plot(np.log(df_pivot['O00391']), df_pivot['updrs_1'])

In [ ]:
plt.plot(df_pivot['Q9NQ79'], df_pivot['updrs_1'])

In [ ]:
plt.plot(np.log(df_pivot['Q9NQ79']), df_pivot['updrs_1'])

## Applying Log Transformation to all the NPxs

In [ ]:
df_pivot.iloc[:, 1:-1] = df_pivot.iloc[:, 1:-1].apply(np.log) 

In [ ]:
df_pivot = df_pivot.replace(float("-inf"), 0)

In [ ]:
df_pivot

In [ ]:
df_pivot = df_pivot.round(4)

## Normalization to (0,1)


In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(df_pivot.iloc[:, 1:-1])

In [ ]:
transformed_values = scaler.transform(df_pivot.iloc[:, 1:-1])

In [ ]:
transformed_values[:, 0]

In [ ]:
type(scaler.transform(df_pivot.iloc[:, 1:-1]))

In [ ]:
plt.hist(df_pivot['O00391'])

In [ ]:
plt.hist(transformed_values[:, 0])

In [ ]:
plt.hist(np.power(transformed_values[:, 0], 4))

In [ ]:
df_pivot.iloc[:, 1:-1] = transformed_values


In [ ]:
df_pivot.iloc[:, 1:-1] = df_pivot.iloc[:, 1:-1].apply(lambda x : np.power(x, 4)) 

In [ ]:
plt.hist(df_pivot['Q9NQ79'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_pivot[df_pivot.columns.difference(['visit_id', 'updrs_1'])], 
                                                    df_pivot[['updrs_1']], test_size=0.1, shuffle=True, random_state=42)

In [ ]:
svr = svm.SVR()

In [ ]:
svr.fit(X_train, y_train.squeeze())

In [ ]:
y_pred = svr.predict(X_test)

In [ ]:
mean_squared_error(y_pred, y_test)

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
plt.hist(y_test)